In [2]:
import numpy as np
import os
import csv
import runtests

In [3]:
def return_one(startdisk, zfsname, csvfilename, ip, skip_num=0):
    with open(csvfilename) as csvfile:
        reader = csv.reader(csvfile, delimiter=",")
        next(reader)
        for _ in range(skip_num):
            next(reader)
        row = next(reader)
        testname, numberdisks, raidmode, raid0, ashift, compression, recordsize, atime, filesize, benchmark, runtime, blocksizes, iodepths, numjobs = row
        print (testname, numberdisks, raidmode, raid0, ashift, compression, recordsize, atime, filesize, benchmark, runtime, blocksizes, iodepths, numjobs)
        print ("++++++++++++++++++++++++++++")
        print (testname)
        print ("++++++++++++++++++++++++++++")
        numberdisks = int(numberdisks)
        raidmode = int(raidmode)
        raid0 = int(raid0)
        return runtests.make_commands(startdisk, numberdisks, zfsname, raidmode, raid0, ashift, compression, recordsize, atime, testname, ip, filesize, benchmark, runtime, blocksizes, iodepths, numjobs)

In [4]:
cmds = return_one("c", "tank", "config.csv", "192.168.169.207", skip_num=8)

z2_three_8_single 24 6 3 12  32m off 1500g True 200 32m 64m 1 2 1 4
++++++++++++++++++++++++++++
z2_three_8_single
++++++++++++++++++++++++++++


In [5]:
for cmd in cmds:
    print(cmd)

mkdir z2_three_8_single
echo -------------------------------
echo make zfs
echo -------------------------------
zpool create tank -o ashift=12 raidz2 sdc sdd sde sdf sdg sdh sdi sdj raidz2 sdk sdl sdm sdn sdo sdp sdq sdr raidz2 sds sdt sdu sdv sdw sdx sdy sdz
echo -------------------------------
echo test zfs
echo -------------------------------
./bench_fio  --type directory --quiet -m write read --loops 1 --target /tank -o z2_three_8_single_zfs -b 32m 64m --iodepth 1 2 --numjobs 1 4 --size 1500g --runtime 200
echo -------------------------------
echo make lustre
echo -------------------------------
mkfs.lustre --backfstype=zfs --fsname=test --index=0 --reformat --mgs --mdt mgsmdt/mgsmdt
mkfs.lustre --backfstype=zfs --fsname=test --index=0 --ost --mgsnode=192.168.169.207@tcp tank/ost0
mount -t lustre mgsmdt/mgsmdt /mnt/mgsmdt
mount -t lustre tank/ost0 /mnt/ost
mount -t lustre 192.168.169.207@tcp:/test /mnt/lustre
echo -------------------------------
echo test lustre
echo --------------

# make sure the change device, name, rw
"fio --directory=" "--direct=1 --rw=write --bs=32m --ioengine=libaio --iodepth=64 --filesize=100g --runtime=120 --time_based --group_reporting --name=throughput-test --eta-newline=1"
"fio --directory=" "--direct=1 --rw=read --bs=32m --ioengine=libaio --iodepth=64 --filesize=100g --runtime=120 --time_based --group_reporting --name=throughput-test --eta-newline=1 --readonly"

In [6]:
fio --directory=/tank --direct=1 --rw=write --bs=32m --ioengine=libaio --iodepth=64 --filesize=30g --numjobs=2 --group_reporting --name=throughput-test --eta-newline=1

./bench_fio --target /tank --type directory -o onedisk_fioplot -b 16m 32m 64m -m write --loops 1 --iodepth 1 2 --numjobs 1 2 --ss iops:10% --ss-ramp 10 --ss-dur 30 --runtime 300 --size 20g

./fio_plot/fio_plot -T "title" -r write -g -t bw -d 1 2 -n 1 2 -i benchmark_script/onedisk_fioplot/tank/32m/

SyntaxError: cannot assign to operator (Temp/ipykernel_8208/472463204.py, line 1)

In [ ]:
./fio-plot/fio_plot/fio_plot -T "bandwidth of write on lustre raidz2(8 drives) given iodepth and numjobs 32m" -i z2_one_8_single_lustre/lustre/16m -g -r write -t bw -d 1 2 4 -n 1 4 8
./fio-plot/fio_plot/fio_plot -T "bandwidth of write on zfs 60 (2x8 drives) given iodepth and numjobs 64m" -i z2_one_8_single_zfs/tank/64m -g -r write -t bw -d 1 2 4 -n 1 4 8
./fio-plot/fio_plot/fio_plot -T "bandwidth of read on zfs raidz2(8 drives) given iodepth and numjobs m" -i z2_one_8_single_zfs/tank/16m -g -r read -t bw -d 1 2 4 -n 1 4 8


In [4]:
import os
import csv

csvfilename = "config.csv"
skip_num = 6

def make_plot_cmd(rw, fs, bs, iodepths, numjobs, testname):
    if fs == "zfs":
        fsdir = '_lustre/lustre/'
    else:
        fsdir = '_zfs/tank/'
    title = 'bandwidth of '+ rw +' on ' + fs + ' ' + testname + ' '+bs
    cmd = './fio-plot/fio_plot/fio_plot -T "' + title + '" -i '+ testname + fsdir +bs+' -g -r '+rw+' -t bw -d '+ iodepths +' -n ' + numjobs + ' --disable-fio-version'
    return cmd, title

with open(csvfilename) as csvfile:
        reader = csv.reader(csvfile, delimiter=",")
        next(reader)
        cmds = []
        for _ in range(skip_num):
            next(reader)
        for row in reader:
            testname, numberdisks, raidmode, raid0, ashift, compression, recordsize, atime, filesize, benchmark, runtime, blocksizes, iodepths, numjobs = row
            rws = ["read", "write"]
            fss = ["zfs", "lustre"]
            blocksizes = blocksizes.split()
            # print (blocksizes)
            graphdir = "graph_" + testname
            cmds.append ("mkdir " + graphdir)
            for bs in blocksizes:
                blockdir = graphdir +"/"+bs
                cmds.append ("mkdir "+ blockdir)
                for fs in fss:
                    for rw in rws:
                        plot_cmd, title = make_plot_cmd (rw, fs, bs, iodepths, numjobs, testname)
                        title = title.replace(" ", "-")
                        cmds.append (plot_cmd)
                        cmds.append ("mv " + title + '* ' + blockdir)
                        # TODO: save plot to specific dir

for cmd in cmds:
    print (cmd)

mkdir graph_z2_one_8_single
mkdir graph_z2_one_8_single/16m
./fio-plot/fio_plot/fio_plot -T "bandwidth of read on zfs z2_one_8_single 16m" -i z2_one_8_single_lustre/lustre/16m -g -r read -t bw -d 1 2 4 -n 1 4 8 --disable-fio-version
mv bandwidth-of-read-on-zfs-z2_one_8_single-16m* graph_z2_one_8_single/16m
./fio-plot/fio_plot/fio_plot -T "bandwidth of write on zfs z2_one_8_single 16m" -i z2_one_8_single_lustre/lustre/16m -g -r write -t bw -d 1 2 4 -n 1 4 8 --disable-fio-version
mv bandwidth-of-write-on-zfs-z2_one_8_single-16m* graph_z2_one_8_single/16m
./fio-plot/fio_plot/fio_plot -T "bandwidth of read on lustre z2_one_8_single 16m" -i z2_one_8_single_zfs/tank/16m -g -r read -t bw -d 1 2 4 -n 1 4 8 --disable-fio-version
mv bandwidth-of-read-on-lustre-z2_one_8_single-16m* graph_z2_one_8_single/16m
./fio-plot/fio_plot/fio_plot -T "bandwidth of write on lustre z2_one_8_single 16m" -i z2_one_8_single_zfs/tank/16m -g -r write -t bw -d 1 2 4 -n 1 4 8 --disable-fio-version
mv bandwidth-of-wri